# New section

### Project Goals

> Find the main drivers of HDB resale prices

> Using dataset from data.gov.sg and also amenities close to a particular area

> Random Forest Model for prediction

### Loading of Data

> NumPy : Library for Numeric Computations in Python  
> Pandas : Library for Data Acquisition and Preparation  
> Matplotlib : Low-level library for Data Visualization  
> Seaborn : Higher-level library for Data Visualization

In [18]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt0
import seaborn as sb
import os
sb.set()
from datetime import datetime

In [19]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


#### Decided to use the 2015 to current time dataset to clean

#### Finding out the shape and dtype of each column from 2015 dataset using `info` and `shape`

In [13]:
resale_1990 = pd.read_csv(os.path.join(project_dir, "content/resale-flat-prices-based-on-approval-date-1990-1999.csv"))
resale_1990.head()

FileNotFoundError: ignored

In [ ]:
resale_2000 = pd.read_csv(os.path.join(project_dir, "content/resale-flat-prices-based-on-approval-date-2000-feb-2012.csv"))
resale_2000.head()

In [ ]:
resale_2012 = pd.read_csv(os.path.join(project_dir, "content/resale-flat-prices-based-on-registration-date-from-mar-2012-to-dec-2014.csv"))
resale_2012.head()

In [ ]:
resale_first = pd.concat([resale_1990,resale_2000,resale_2012],axis=0,sort=False)
resale_first

In [ ]:
resale_first.info()

In [ ]:
count = 0
remaining_list = []
for i in resale_first.month:
  year = int(i[:4])
  remaining = 99 - (year - resale_first.lease_commence_date.iloc[count])
  remaining_list.append(remaining)
  count+=1

resale_first["remaining_lease"] = remaining_list
resale_first.head()

In [ ]:
resale_2015 = pd.read_csv(os.path.join(project_dir, "content/resale-flat-prices-based-on-registration-date-from-jan-2015-to-dec-2016.csv"))
resale_2015.head()

In [ ]:
resale_2015.shape

In [ ]:
resale_2015.info() # we know that there are no NaN values in any column

#### Finding out the shape and dtype of each column from 2017 dataset using `info` and `shape`

In [ ]:
resale_current = pd.read_csv(os.path.join(project_dir,"content/resale-flat-prices-based-on-registration-date-from-jan-2017-onwards.csv"))
resale_current.head()

In [ ]:
resale_current.shape

In [ ]:
resale_current.info() #we know there are no NaN values as well in all the columns

In [ ]:
# Concatenate both the datasets
resale_total = pd.concat([resale_2015,resale_current],axis=0,sort=False)
resale_first = resale_first.reindex(columns=resale_total.columns)
resale_total = pd.concat([resale_first,resale_total],axis=0,sort=False)
resale_total.month = pd.to_datetime(resale_total.month) #to date time

resale_total.info()

### Data Cleaning
*   Change names of the duplicate flat models
*   Adjust resale prices according to inflation 
*   Converting remaining_lease to years









In [ ]:
resale_total

#### Converting remaining_lease to years

In [ ]:
def convert(lease):
  if type(lease) is str:
    length = [int(years) for years in lease.split() if years.isdigit()]
    if len(length)>1:
      final = length[0] + (length[1])/12
    else:
      final = length[0]
    return final
  else:
    return lease

In [ ]:
resale_total.remaining_lease = resale_total.remaining_lease.apply(lambda x: convert(x))
resale_total

In [ ]:
resale_total.info()

#### Changing duplicate names of flat models

In [ ]:
resale_total.flat_model.unique()

In [ ]:
resale_total.flat_model.value_counts()

In [ ]:
'''rename Improved-Maisonette to Executive Maisonette
Premium Maisonette to Executive Maisonette
Model A-Maisonette to Maisonette
Premium Apartment Loft to Premium Apartment
Type S1 and S2 to TypeS1S2'''

def change_model(model):
  if model == "Improved-Maisonette" or model == "Premium Maisonette" or model == "IMPROVED-MAISONETTE":
    rename = "Executive Maisonette"
  elif model == "Model A-Maisonette" or model=="MODEL A-MAISONETTE" or model=="MAISONETTE":
    rename = "Maisonette"
  elif model == "Premium Apartment Loft" or model=="PREMIUM APARTMENT":
    rename = "Premium Apartment"
  elif model == "Type S1" or model == "Type S2":
    rename = "Type S1S2"
  elif model == "DBSS":
    rename = "DBSS"
  else:
    complete = ""
    split = model.split()
    if len(split) == 1:
      return model[0] + model[1:].lower()
    else:
      for i in split:
        complete += (i[0] + i[1:].lower() + " ")
      if complete[-1]==" ":
        complete = complete[0:-1]
      return complete
  return rename

In [ ]:
resale_total.flat_model = resale_total.flat_model.apply(lambda x:change_model(x))
resale_total.flat_model.unique()

In [ ]:
resale_total.flat_model.value_counts()

#### Adjusting Resale prices using inflation using CPI

In [ ]:
# https://www.singstat.gov.sg/find-data/search-by-theme/economy/prices-and-price-indices/latest-data
CPI = pd.read_excel(os.path.join(project_dir, 'content/CPI_2022.xlsx'))
CPI.head()

In [ ]:
CPI_rate = pd.DataFrame(CPI.loc[0,'1990 Jan':]).reset_index()
CPI_rate.rename(columns={'index':'month',0:'CPI rates'},inplace=True)
CPI_rate

In [ ]:
CPI_rate.info()

In [ ]:
# changing the month format
CPI_rate.month = pd.to_datetime(CPI_rate.month,format='%Y %b')
CPI_rate['CPI rates'] = pd.to_numeric(CPI_rate['CPI rates'])
CPI_rate.head()

In [ ]:
CPI_rate.info()

In [ ]:
#merging and then calculating adjusted resale price
resale_total = resale_total.merge(CPI_rate,on='month',how='left')
resale_total

In [ ]:
resale_total.info()

In [ ]:
real_price=[]
count = 0
for i in resale_total['CPI rates']:
  if np.isnan(i):
    real_price.append(resale_total['resale_price'].iloc[count])
  else:
    real_price.append((resale_total['resale_price'].iloc[count]/i)*100)
  count+=1

In [ ]:
resale_total['real_price'] = real_price
resale_total

## Exploratory Data Analysis

*   Kim's part
*   Distance to nearest MRT
*   Number of ammementies nearby


# Kim's EDA

In [ ]:
resale_total.info()

# analysis of catagorical variables

In [ ]:
resaleCat_data = pd.DataFrame(resale_total[['town', 'flat_type', 'storey_range','flat_model']])
resaleCat_data.head()

In [ ]:
resaleCat_data = resaleCat_data.astype('category')

In [ ]:
resaleCat_data.info()

In [ ]:
resaleCat_data.describe()

In [ ]:
resaleCat_data["town"].nunique() #27 unique towns

In [ ]:
resaleCat_data["town"].value_counts()

In [ ]:
# Modified catplot for counts sorted by the counts
sb.catplot(y = 'town', data = resaleCat_data, 
           kind = "count", 
           height = 8, 
           order = resaleCat_data['town'].value_counts().index)

In [ ]:
# Modified catplot for counts sorted by the counts
sb.catplot(y = 'flat_type', data = resaleCat_data, 
           kind = "count", 
           height = 8, 
           order = resaleCat_data['flat_type'].value_counts().index)

In [ ]:
# Modified catplotstorey_rangefor counts sorted by the counts
sb.catplot(y = 'storey_range', data = resaleCat_data, 
           kind = "count", 
           height = 8, 
           order = resaleCat_data['storey_range'].value_counts().index)

In [ ]:
# Modified catplotstorey_rangefor counts sorted by the counts
sb.catplot(y = 'flat_model', data = resaleCat_data, 
           kind = "count", 
           height = 8, 
           order = resaleCat_data['flat_model'].value_counts().index)

In [ ]:
# Add SalePrice to the dataframe
resaleCat_data = pd.concat([resaleCat_data, resale_total["resale_price"]], axis = 1).reindex(resaleCat_data.index)
resaleCat_data.head()

In [ ]:
import matplotlib.pyplot as plt # we only need pyplot
sb.set() # set the default Seaborn style for graphics

In [ ]:
f = plt.figure(figsize=(16, 8))
sb.boxplot(x = 'town', y = 'resale_price', data = resaleCat_data,order = resaleCat_data.groupby('town')['resale_price'].median().sort_values('town').index)
# Tilt the x-axis labels for better readability
plt.xticks(rotation=45);

In [ ]:
f = plt.figure(figsize=(16, 8))
sb.boxplot(x = 'flat_type', y = 'resale_price', data = resaleCat_data, 
           order = resaleCat_data.groupby('flat_type')['resale_price'].median().sort_values().index)
# Tilt the x-axis labels for better readability
plt.xticks(rotation=45);

In [ ]:
f = plt.figure(figsize=(16, 8))
sb.boxplot(x = 'storey_range', y = 'resale_price', data = resaleCat_data, 
           order = resaleCat_data.groupby('storey_range')['resale_price'].median().sort_values().index)
# Tilt the x-axis labels for better readability
plt.xticks(rotation=45);

In [ ]:
f = plt.figure(figsize=(16, 8))
sb.boxplot(x = 'flat_model', y = 'resale_price', data = resaleCat_data, 
           order = resaleCat_data.groupby('flat_model')['resale_price'].median().sort_values().index)
# Tilt the x-axis labels for better readability
plt.xticks(rotation=45);

We can see highest variation of resale price is caused by flat model

Now lets analyse numerical variables

# Analysis of Numeric Variables

In [ ]:
resaleNum_data = pd.DataFrame(resale_total[['floor_area_sqm', 'lease_commence_date', 'remaining_lease','CPI rates']])
resaleNum_data.head()

In [ ]:
resaleNum_data.describe()

In [ ]:
# Draw the distributions of all variables
f, axes = plt.subplots(4, 3, figsize=(18, 20))

count = 0
for var in resaleNum_data:
    sb.boxplot(data = resaleNum_data[var], orient = "h", ax = axes[count,0])
    sb.histplot(data = resaleNum_data[var], ax = axes[count,1])
    sb.violinplot(data = resaleNum_data[var], orient = "h", ax = axes[count,2])
    count += 1

In [ ]:
# Calculate the quartiles
Q1 = resaleNum_data.quantile(0.25)
Q3 = resaleNum_data.quantile(0.75)

# Rule to identify outliers
rule = ((resaleNum_data < (Q1 - 1.5 * (Q3 - Q1))) | (resaleNum_data > (Q3 + 1.5 * (Q3 - Q1))))

# Count the number of outliers
rule.sum()

In [ ]:
resaleNum_data.skew()

In [ ]:
# Add resale price to the dataframe
resaleNum_data = pd.concat([resaleNum_data, resale_total["resale_price"]], axis = 1).reindex(resaleNum_data.index)

# Correlation Matrix
print(resaleNum_data.corr())

# Heatmap of the Correlation Matrix
f = plt.figure(figsize=(12, 12))
sb.heatmap(resaleNum_data.corr(), vmin = -1, vmax = 1, linewidths = 1,
           annot = True, fmt = ".2f", annot_kws = {"size": 18}, cmap = "RdBu")

# CPI rates has highest correlation with the resale price and remaining lease has the lowest correlation with the resale price. It is interesting and unexpecting to see the low correlation between resale price and remaining lease as we usually expect a house with a longer lease to have higher value as we can stay in the house longer. On the other hand, the fact that CPI rate having the highest correlation with the resale price, as increasing CPI leads to higher inflation, which would eventually lead to rise in resale prices of the houses. 


In [ ]:
sb.jointplot(data = resaleNum_data, x = "floor_area_sqm", y = "resale_price", height = 12)

In [ ]:
sb.jointplot(data = resaleNum_data, x = "remaining_lease", y = "resale_price", height = 12)

In [ ]:
sb.jointplot(data = resaleNum_data, x = "lease_commence_date", y = "resale_price", height = 12)

In [ ]:
sb.jointplot(data = resaleNum_data, x = "CPI rates", y = "resale_price", height = 12)

### Adding distance to nearest MRT

Given the reliability of our public transport and high cost of owning a car, Singaporeans are rather dependent on MRTs. Thus distance to the nearest MRT station is often one factor that homeowners consider when purchasing a house. We will be using OneMap API to calculate the distance between a flat and the nearest MRT station to analyse this effect

In [ ]:
resale_total.info()

In [ ]:
resale_total

In [ ]:
import pandas as pd
import numpy as np
import requests
import json

In [ ]:
## Function for getting postal code, geo coordinates of addresses

def find_postal(lst, filename):
    '''With the block number and street name, get the full address of the hdb flat,
    including the postal code, geogaphical coordinates (lat/long)'''
    
    for index,add in enumerate(lst):
        # Do not need to change the URL
        url= "https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal="+ add        
        print(index,url)
        
        # Retrieve information from website
        response = requests.get(url)
        try:
            data = json.loads(response.text) 
        except ValueError:
            print('JSONDecodeError')
            pass
    
        temp_df = pd.DataFrame.from_dict(data["results"])
        # The "add" is the address that was used to search in the website
        temp_df["address"] = add
        
        # Create the file with the first row that is read in 
        if index == 0:
            file = temp_df
        else:
            file = file.append(temp_df)
    file.to_csv(filename + '.csv')

    ## Function for getting closest distance of each location from a list of amenities location

from geopy.distance import geodesic

def find_nearest(house, amenity, radius=2):
    """
    this function finds the nearest locations from the 2nd table from the 1st address
    Both are dataframes with a specific format:
        1st column: any string column ie addresses taken from the "find_postal_address.py"
        2nd column: latitude (float)
        3rd column: longitude (float)
    Column name doesn't matter.
    It also finds the number of amenities within the given radius (default=2)
    """
    results = {}
    # first column must be address
    for index,flat in enumerate(house.iloc[:,0]):
        
        # 2nd column must be latitude, 3rd column must be longitude
        flat_loc = (house.iloc[index,1],house.iloc[index,2])
        flat_amenity = ['','',100,0]
        for ind, eachloc in enumerate(amenity.iloc[:,0]):
            amenity_loc = (amenity.iloc[ind,1],amenity.iloc[ind,2])
            distance = geodesic(flat_loc,amenity_loc)
            distance = float(str(distance)[:-3]) # convert to float

            if distance <= radius:   # compute number of amenities in 2km radius
                flat_amenity[3] += 1

            if distance < flat_amenity[2]: # find nearest amenity
                flat_amenity[0] = flat
                flat_amenity[1] = eachloc
                flat_amenity[2] = distance

        results[flat] = flat_amenity
    return results

def dist_from_location(house, location):
    """
    this function finds the distance of a location from the 1st address
    First is a dataframe with a specific format:
        1st column: any string column ie addresses taken from the "find_postal_address.py"
        2nd column: latitude (float)
        3rd column: longitude (float)
    Column name doesn't matter.
    Second is tuple with latitude and longitude of location
    """
    results = {}
    # first column must be address
    for index,flat in enumerate(house.iloc[:,0]):
        
        # 2nd column must be latitude, 3rd column must be longitude
        flat_loc = (house.iloc[index,1],house.iloc[index,2])
        flat_amenity = ['',100]
        distance = geodesic(flat_loc,location)
        distance = float(str(distance)[:-3]) # convert to float
        flat_amenity[0] = flat
        flat_amenity[1] = distance
        results[flat] = flat_amenity
    return results